# Parte I – Resolvendo um Problema Linearmente Separável

### Bibliotecas

A primeira célula do notebook será reservada para importação de bibliotecas.

In [1]:
import pandas as pd
import numpy as np
import struct, random, copy
import matplotlib.pyplot as plt

In [2]:
dt = np.dtype([('time', [('min', np.int64), ('sec', np.int64)]), ('temp', float)])
df = np.fromfile('dataframe/dataAll.txt', dtype=dt)

##### 1. As equipes devem utilizar a função de ativação degrau com θ = 0 (Fílip Anselmo);

In [1]:
def degrau(u):
    if(u >= 0):
        return 1
    else:
        return 0

##### 2. O valor da taxa de aprendizado deve ser igual a η = 0,1;

In [3]:
learning_rate = 0.1
print("learning_rate: ", learning_rate)

learning_rate:  0.1


##### 3. O vetor inicial de pesos deve ter seus valores inicializados conforme uma variável aleatória de distribuição uniforme no intervalo, isto é, wi ∼ U(−0,5, + 0,5). O vetor inicial de pesos deve ser impresso no início da execução do algoritmo; (Erik Atilio)

In [4]:
weights_initial_vector = []

for i in range(int(df.shape[0])):
    weights_initial_vector.append(round(random.uniform(-0.5, 0.5), 1))

##### 4. A cada época deve ser indicado o número de ajustes feitos no vetor de pesos;

##### 5. Sempre que o vetor de pesos for ajustado, este deve ser impresso;

##### 6. O algoritmo deve executar até a convergência, isto é, até que não haja erros para todos os exemplos presentes no conjunto de treinamento;

#### Definição da class perceptron orientada a objeto

In [5]:
class Perceptron:

    def __init__(self, amostras, saidas, taxa_aprendizado=0.1, epocas=1000, limiar=-1):

        self.amostras = amostras # todas as amostras
        self.saidas = saidas # saídas respectivas de cada amostra
        self.taxa_aprendizado = taxa_aprendizado # taxa de aprendizado (entre 0 e 1)
        self.epocas = epocas # número de épocas
        self.limiar = limiar # limiar
        self.num_amostras = len(amostras) # quantidade de amostras
        self.num_amostra = len(amostras[0]) # quantidade de elementos por cada amostra
        self.pesos = [] # vetor dos pesos


    # função utilizada para treinar a rede
    def treinar(self):

        # adiciona -1 para cada amostra
        for amostra in self.amostras:
            amostra.insert(0, -1)

        # inicia o vetor de pesos com valores aleatórios pequenos
        for i in range(self.num_amostra):
            self.pesos.append(random.random())

        # insere o limiar no vetor de pesos
        self.pesos.insert(0, self.limiar)

        num_epocas = 0 # inicia o contador de épocas
        
        while True:

            erro = False # erro inicialmente inexiste

            # para todas as amostras de treinamento
            for i in range(self.num_amostras):
                
                u = 0
                '''
                    realiza o somatório, o limite (self.num_amostra + 1) 
                    é porque foi inserido o -1 em cada amostra
                '''
                for j in range(self.num_amostra + 1):
                    u += self.pesos[j] * self.amostras[i][j]
                    
                # obtém a saída da rede utilizando a função de ativação
                y = self.sinal(u)

                # verifica se a saída da rede é diferente da saída desejada
                if y != self.saidas[i]:

                    # calcula o erro: subtração entre a saída desejada e a saída da rede
                    erro_aux = self.saidas[i] - y

                    # faz o ajuste dos pesos para cada elemento da amostra
                    for j in range (self.num_amostra + 1):
                        self.pesos[j] = self.pesos[j] + self.taxa_aprendizado * erro_aux * self.amostras[i][j]

                    erro = True # se entrou, é porque o erro ainda existe
                    
            num_epocas += 1 # incrementa o número de épocas

            # critério de parada é pelo número de épocas ou se não existir erro
            if num_epocas > self.epocas or not erro:
                break


    # função utilizada para testar a rede
    # recebe uma amostra a ser classificada e os nomes das classes
    # utiliza função sinal, se é -1 então é classe1, senão é classe2
    def testar(self, amostra, classe1, classe2):

        # insere o -1
        amostra.insert(0, -1)

        '''
            utiliza-se o vetor de pesos ajustado
            durante o treinamento da rede
        '''
        u = 0
        for i in range(self.num_amostra + 1):
            u += self.pesos[i] * amostra[i]

        # calcula a saída da rede
        y = self.sinal(u)

        # verifica a qual classe pertence
        if y == -1:
            print('A amostra pertence a classe %s' % classe1)
        else:
            print('A amostra pertence a classe %s' % classe2)


    def degrau(self, u):
        return 1 if u >= 0 else 0


    def sinal(self, u): # é a mesma função degrau bipolar
        return 1 if u >= 0 else -1